In [12]:
#pip install beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.7 MB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.3 MB/s eta 0:00:00


 The fist part scrapes all the contents from each url within the webpage

In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
'''
This is a function to get all the links that contains the content of the txt books.
The max depth of link within link is 2, to retrieve all links, we check all link start with 'http://www.authorama.com/'
'''
def get_all_links(url, depth=0, max_depth=1):
    if depth > max_depth:
        return []

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        absolute_url = urljoin(url, href)
        if absolute_url.startswith('http://www.authorama.com/'):
            links.append(absolute_url)
            nested_links = get_all_links(absolute_url, depth=depth+1, max_depth=max_depth)
            links.extend(nested_links)

    return links

url = 'http://www.authorama.com/'
book_links = get_all_links(url)

#print(book_links)


In [20]:
import os

def get_book_text(book_url, output_dir):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Create a unique filename for each book chapter based on its URL
    filename = os.path.join(output_dir, book_url.split('/')[-1] + '.txt')

    # Save the content to the file directly, without storing it in memory
    with open(filename, 'w') as file:
        for paragraph in soup.find_all('p'):
            file.write(paragraph.text + '\n')


output_dir = 'your local path'
for book_link in book_links:
    get_book_text(book_link, output_dir)
